In [53]:
import pandas as pd

data = pd.read_json("./05_clean_data.json.zip")
data

,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime_baseline,baseline_sample_size,path_to_baseline_sample,parameters,instance_name,lower_bound,upper_bound,iteration_info,time_used_by_yasa,timelimit_for_samplns,samplns_used,runtime_samplns,time_limit
0,1050,996,YASA,t2_m1_null,2.278,39,2023-03-01_13-51-03/38_1_7_5_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",busybox_2020-12-16_21-53-05,18,23,"[{'nbrhd_tuples': 222, 'nbrhd_confs': 4, 'iter...",2.278,897.722,True,909.288080,900
1,10,17,YASA,t2_m1_null,0.199,7,2023-03-01_13-51-03/3_1_7_5_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",email,6,6,"[{'nbrhd_tuples': 70, 'nbrhd_confs': 7, 'itera...",0.199,899.801,True,0.172952,900
2,580,879,YASA,t2_m1_null,1.438,149,2023-03-01_13-51-03/34_1_7_1_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",WaterlooGenerated,82,82,"[{'nbrhd_tuples': 248, 'nbrhd_confs': 36, 'ite...",1.438,898.562,True,637.711078,900
3,19,27,YASA,t2_m1_null,0.207,12,2023-03-01_13-51-03/7_1_7_5_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",FeatureIDE,7,8,"[{'nbrhd_tuples': 214, 'nbrhd_confs': 8, 'iter...",0.207,899.793,True,905.908822,900
4,19,27,YASA,t2_m1_null,0.208,12,2023-03-01_13-51-03/7_1_7_4_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",FeatureIDE,7,8,"[{'nbrhd_tuples': 215, 'nbrhd_confs': 9, 'iter...",0.208,899.792,True,905.727147,900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,101,203,YASA,t2_m1_null,0.288,27,2023-03-01_13-51-03/19_1_7_1_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",Violet,14,17,"[{'nbrhd_tuples': 178, 'nbrhd_confs': 7, 'iter...",0.288,899.712,True,923.982945,900
226,366,627,YASA,t2_m1_null,0.597,27,2023-03-01_13-51-03/30_1_7_5_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",DMIE,14,17,"[{'nbrhd_tuples': 241, 'nbrhd_confs': 5, 'iter...",0.597,899.403,True,909.323218,900
227,258,1542,YASA,t2_m1_null,1.152,224,2023-03-01_13-51-03/25_1_7_3_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",fiasco_2020-12-01_14-09-14,196,196,"[{'nbrhd_tuples': 236, 'nbrhd_confs': 19, 'ite...",1.152,898.848,True,686.337217,900
228,39,77,YASA,t2_m1_null,0.202,17,2023-03-01_13-51-03/14_1_7_3_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",SortingLine,9,9,"[{'nbrhd_tuples': 206, 'nbrhd_confs': 12, 'ite...",0.202,899.798,True,66.414268,900


In [54]:
# calculate the best known lower bound
best_lower_bound = (
    data.groupby(["instance_name"])["lower_bound"]
    .max()
    .reset_index()
    .rename(columns={"lower_bound": "best_lower_bound"})
)
data = data.merge(best_lower_bound, on="instance_name")

In [56]:
# calculate the best upper bound
best_upper_bound = (
    data.groupby(["instance_name"])["upper_bound"]
    .min()
    .reset_index()
    .rename(columns={"upper_bound": "best_upper_bound"})
)
data = data.merge(best_upper_bound, on="instance_name")

In [64]:
# calculate best baseline algorithm sample
baseline_data = pd.read_json("./00_simple_baseline_data.json.zip")
best_baseline = (
    baseline_data.groupby(["instance_name"])[["sample_size"]]
    .min()
    .reset_index()
    .rename(columns={"sample_size": "best_baseline_sample_size"})
)
data = data.merge(best_baseline, on="instance_name", how="left")

In [69]:
t = data.groupby("instance_name")[
    [
        "num_vars",
        "num_clauses",
        "best_baseline_sample_size",
        "baseline_sample_size",
        "upper_bound",
        "lower_bound",
        "best_upper_bound",
        "best_lower_bound",
    ]
].mean()
t.sort_values(["num_vars", "num_clauses"], inplace=True)
# checking consistency is important.
assert (t["best_lower_bound"] <= t["best_baseline_sample_size"]).all()
assert (t["best_upper_bound"] >= t["best_lower_bound"]).all()
t

,num_vars,num_clauses,best_baseline_sample_size,baseline_sample_size,upper_bound,lower_bound,best_upper_bound,best_lower_bound
instance_name,,,,,,,,
calculate,9.0,15.0,9.0,10.0,5.0,5.0,5.0,5.0
lcm,9.0,16.0,8.0,8.0,6.0,6.0,6.0,6.0
email,10.0,17.0,6.0,7.0,6.0,6.0,6.0,6.0
ChatClient,14.0,20.0,8.0,8.0,7.0,7.0,7.0,7.0
toybox_2006-10-31_23-30-06,16.0,13.0,9.0,10.0,8.0,8.0,8.0,8.0
car,16.0,33.0,6.0,7.0,5.0,5.0,5.0,5.0
FeatureIDE,19.0,27.0,11.0,12.0,8.0,7.0,8.0,7.0
FameDB,22.0,40.0,8.0,8.0,8.0,8.0,8.0,8.0
APL,23.0,35.0,9.0,10.0,7.0,7.0,7.0,7.0
